In [19]:
%store -r __imp

In [20]:
__imp

The libraries have been loaded!


In [21]:
import argparse
import io
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
from google.oauth2 import service_account
import glob 
import time
import os


In [22]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(language_code='pt-BR')
    
    print('Tentando')
    operation = client.long_running_recognize(config, audio)

    print('Waiting for operation to complete...')
    try:
        response = operation.result()
    except Exception as e:
        response = operation.result()
        pass
    retorno = [{'fala' : result.alternatives[0].transcript,'confianca' : result.alternatives[0].confidence} for result in response.results]
    return retorno

In [ ]:
#r = transcribe_file('C:\\Work\\Python\\religious_work\\Gravação.m4a')
import re
from collections import deque
import time

def trata_arquivo_pregacao(nm_arq):
    try:    
        dt_ini = time.time()
        localtime = time.asctime( time.localtime(dt_ini) )
        print('Lendo arquivo %s %s' %(nm_arq,localtime))
        lst_texto = transcribe_gcs('gs://pregacoes/' + nm_arq)
        #lst_texto = [{'fala' : 'ola','confianca' : 0.83}]
        df_texto = pd.DataFrame(lst_texto)

        df_texto['pregador'] = re.sub('[^A-Za-z]', '', nm_arq[:-3])
        df_texto['arq_ogm'] = nm_arq
        try:
            pd.read_csv('texto_pregacoes.csv').append(df_texto).to_csv('texto_pregacoes.csv',index=False)
        except Exception as e:
            df_texto.to_csv('texto_pregacoes.csv',index=False)
            pass
        print('Tempo gasto %.2f min.' %((time.time() - dt_ini)/60))
        return {'arq' : nm_arq,'rstd' : True, 'erro' : ''}
    except Exception as e:
        return {'arq' : nm_arq,'rstd' : False, 'erro' : str(e)}
        pass

arqs_bucket = [a.split('\\')[-1] for a in glob.glob('C:\\Users\\Italo Dell\\Music\\Wav\\*.wav')]
print('%i arquivos ' %len(arqs_bucket))
resultado = list(map(trata_arquivo_pregacao,arqs_bucket))
df_rstd = pd.DataFrame(resultado)
df_rstd.describe()
if df_rstd[~df_rstd.rstd].shape[0] > 0: 
    print('%i arquivos com erros' %df_rstd[~df_rstd.rstd].shape[0])
    resultado = list(map(trata_arquivo_pregacao,df_rstd.arq.values))
    print('Segundo resultado')    
    print(pd.DataFrame(resultado).describe())    


70 arquivos 
Lendo arquivo 1_4902614922531176478_joaquim.wav Tue Apr 24 22:59:37 2018
Tentando
Waiting for operation to complete...
Tempo gasto 12.56 min.
Lendo arquivo 1_4902614922531176482_martinez.wav Tue Apr 24 23:12:11 2018
Tentando
Waiting for operation to complete...
Tempo gasto 10.23 min.
Lendo arquivo 1_4945009032881504284_marçola.wav Tue Apr 24 23:22:25 2018
Tentando
Lendo arquivo 1_4967837943700914212_valmir.wav Tue Apr 24 23:22:26 2018
Tentando
Waiting for operation to complete...
Tempo gasto 18.39 min.
Lendo arquivo 1_4978823387712323621_martinez.wav Tue Apr 24 23:40:50 2018
Tentando
Waiting for operation to complete...
Lendo arquivo 1_5017055962515636305_martinez.wav Wed Apr 25 00:00:35 2018
Tentando
Waiting for operation to complete...
Tempo gasto 14.90 min.
Lendo arquivo 1_5039674067750223891_messias.wav Wed Apr 25 00:15:29 2018
Tentando
Waiting for operation to complete...
Tempo gasto 7.58 min.
Lendo arquivo 1_5040052750721744933_marciel.wav Wed Apr 25 00:23:04 2018
Te